<a href="https://colab.research.google.com/github/MarianoChic09/MSc-AI-taller-de-deep-learning/blob/main/Ejemplo%20sencillo%20de%20atenci%C3%B3n%20(con%20no%20peek%20mask).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn.functional as F

# Mecanismo de atencion

![Attention diagram](https://miro.medium.com/max/336/1*15E9qKg9bKnWdSRWCyY2iA.png)
![Attention equation](https://miro.medium.com/max/1068/1*evdACdTOBT5j1g1nXialBg.png)

Ignorando la noción de multihead attention y el batch size (por ahora) podemos imaginar un dato de dimensiones (seq_len, d_model)

Dicho dato, es procesado por 3 perceptrones independientes que llamaremos pQ, pK y pV, con dimensión de entrada y salida d_model, a efectos de simplificar el ejemplo asumiremos que son la función identidad

In [2]:
seq_len = 5
d_model = 10
dato = torch.tensor(range(d_model*seq_len), dtype=torch.float32).view(seq_len, d_model)/100
print(dato)
print(dato.shape)

tensor([[0.0000, 0.0100, 0.0200, 0.0300, 0.0400, 0.0500, 0.0600, 0.0700, 0.0800,
         0.0900],
        [0.1000, 0.1100, 0.1200, 0.1300, 0.1400, 0.1500, 0.1600, 0.1700, 0.1800,
         0.1900],
        [0.2000, 0.2100, 0.2200, 0.2300, 0.2400, 0.2500, 0.2600, 0.2700, 0.2800,
         0.2900],
        [0.3000, 0.3100, 0.3200, 0.3300, 0.3400, 0.3500, 0.3600, 0.3700, 0.3800,
         0.3900],
        [0.4000, 0.4100, 0.4200, 0.4300, 0.4400, 0.4500, 0.4600, 0.4700, 0.4800,
         0.4900]])
torch.Size([5, 10])


In [3]:
Q = K = V = dato

El primer paso de la atención es multiplicar Q por K transpuesta:

In [12]:
QKt = torch.matmul(Q, K.transpose(-2, -1))
QKt

tensor([[0.0285, 0.0735, 0.1185, 0.1635, 0.2085],
        [0.0735, 0.2185, 0.3635, 0.5085, 0.6535],
        [0.1185, 0.3635, 0.6085, 0.8535, 1.0985],
        [0.1635, 0.5085, 0.8535, 1.1985, 1.5435],
        [0.2085, 0.6535, 1.0985, 1.5435, 1.9885]])

Si pensamos como marco de referencia nuestro dato, e interpretamos su ultimo valor como el encoding asociado al símbolo de padding sería deseable que la posición 5 del valor retornado por la atención no tenga aporte.

Para ello podemos crear una máscara de padding con un 0 en la posición 5.

In [13]:
mask = torch.tensor([1,1,1,1,0])

Aplicar la máscara en este caso es sencillo

In [14]:
def build_nopeak_mask(size):
  nopeak_mask = torch.triu(torch.ones(size,size)).transpose(0,1)
  return nopeak_mask.unsqueeze(0).type(torch.float32)

In [21]:
no_peak = build_nopeak_mask(5)

In [26]:
no_peak_mask_applied = pad_mask_applied.masked_fill(no_peak ==0, -1e-9).squeeze(0)

In [27]:
no_peak_mask_applied

tensor([[ 2.8500e-02, -1.0000e-09, -1.0000e-09, -1.0000e-09, -1.0000e-09],
        [ 7.3500e-02,  2.1850e-01, -1.0000e-09, -1.0000e-09, -1.0000e-09],
        [ 1.1850e-01,  3.6350e-01,  6.0850e-01, -1.0000e-09, -1.0000e-09],
        [ 1.6350e-01,  5.0850e-01,  8.5350e-01,  1.1985e+00, -1.0000e-09],
        [ 2.0850e-01,  6.5350e-01,  1.0985e+00,  1.5435e+00, -1.0000e+09]])

En el tensor resultante, las filas (i) representan palabras, y las columnas (j) también, donde se expresa que para cada i hay una relación con j, donde para el caso j = 4 este debe ser ignorado por los i pues este elemento corresponde al padding.

El paso siguiente es ponderar estos resultados en un valor entre 0 y 1, pudiendo interpretarse dicho valor como la importancia de la palbra j para la palabra i.

In [28]:
importances = F.softmax(no_peak_mask_applied)

importances

<ipython-input-28-6f9094d5a48e>:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  importances = F.softmax(no_peak_mask_applied)


tensor([[0.2046, 0.1989, 0.1989, 0.1989, 0.1989],
        [0.2023, 0.2339, 0.1880, 0.1880, 0.1880],
        [0.1759, 0.2247, 0.2871, 0.1562, 0.1562],
        [0.1239, 0.1750, 0.2471, 0.3488, 0.1052],
        [0.1137, 0.1774, 0.2769, 0.4320, 0.0000]])

La siguiente matriz nos dice por ejemplo que el aporte al valor final para la palabra 2 dado por la palabra 3 es aproximadamente 35%

In [ ]:
import numpy as np
np.array(np.round(importances, decimals = 2)*100, dtype = "int32")

array([[23, 24, 26, 27,  0],
       [20, 23, 27, 31,  0],
       [17, 21, 27, 35,  0],
       [14, 20, 28, 39,  0],
       [11, 18, 28, 43,  0]], dtype=int32)

Finalmente debemos obtener un resultado para cada palabra de entrada, esto es, un vector que resulta de operar V, con la matriz de importancias.

In [24]:

importances = F.softmax(no_peak_mask_applied)

importances

<ipython-input-24-6f9094d5a48e>:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  importances = F.softmax(no_peak_mask_applied)


tensor([[[1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.],
         [1., 1., 1., 1., 1.]]])

In [ ]:
result = torch.matmul(importances, V)
result

tensor([[0.1556, 0.1656, 0.1756, 0.1856, 0.1956, 0.2056, 0.2156, 0.2256, 0.2356,
         0.2456],
        [0.1680, 0.1780, 0.1880, 0.1980, 0.2080, 0.2180, 0.2280, 0.2380, 0.2480,
         0.2580],
        [0.1801, 0.1901, 0.2001, 0.2101, 0.2201, 0.2301, 0.2401, 0.2501, 0.2601,
         0.2701],
        [0.1917, 0.2017, 0.2117, 0.2217, 0.2317, 0.2417, 0.2517, 0.2617, 0.2717,
         0.2817],
        [0.2027, 0.2127, 0.2227, 0.2327, 0.2427, 0.2527, 0.2627, 0.2727, 0.2827,
         0.2927]])

In [ ]:
result.shape

torch.Size([5, 10])

Finalmente tenemos una salida del mismo tamaño que la entrada. Todas estas operaciones se pueden realizar a la misma vez para un batch.